In [7]:
# importing modules

import pyodbc
import pandas as pd

# import variables
from variable import server, username, password, database_name, table_staging_raw, table_staging_clean, path_to_file
from haversine import haversine, Unit



In [8]:
# connecting to sql server 

connection = None
try:
    connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' +
                                server+';UID='+username+';PWD=' + password+';Database=' + database_name)

    print('Connected to SQL Server Successfully')
except:
    print('Connection failed to SQL Server')

if connection is not None:
    connection.autocommit = True

cur = connection.cursor()

Connected to SQL Server Successfully


In [33]:


df_mushrooms = pd.read_sql_query("SELECT [cap-shape], [cap-color], odor, [gill-size], [gill-color], [stalk-color-above-ring], [veil-color], [ring-type], [spore-print-color], population, habitat, lat, lon FROM {} ".format(table_staging_clean),connection)
#print(sql_query)

print('dataframe loaded with data')

dataframe loaded with data


In [24]:
df_mushroom_group_by_attributes = pd.read_sql_query("SELECT  [cap-shape], [cap-color], odor, [gill-size], [gill-color], [stalk-color-above-ring], [veil-color], [ring-type], [spore-print-color], population, habitat, count(*) count_of_species_per_attributes_grouping FROM {} GROUP BY [cap-shape], [cap-color], [odor], [gill-size],[gill-color], [stalk-color-above-ring], [veil-color], [ring-type], [spore-print-color], [population], [habitat]".format(table_staging_clean),connection)

print('SQL Grouping applied to SQL dataset and loaded to Dataframe')



SQL Grouping applied to SQL dataset and loaded to Dataframe


In [27]:
index = df_mushroom_group_by_attributes.index
number_of_diff_species_of_mushroom = len(index)

print('Q1-How many different species of mushroom are there, if a species is identified by the attributes 1-20?')
print('number_of_diff_species_of_mushroom: '  + str(number_of_diff_species_of_mushroom))



Q1-How many different species of mushroom are there, if a species is identified by the attributes 1-20?
number_of_diff_species_of_mushroom: 170


In [29]:
df_mushrooms.head(4)

,cap-shape,cap-color,odor,gill-size,gill-color,stalk-color-above-ring,veil-color,ring-type,spore-print-color,population,habitat,lat,lon
0,p,s,t,c,brown,s,white,pendant,black,scattered,urban,46.4504,62.4556
1,e,s,t,c,buff,s,white,pendant,brown,numerous,grasses,37.6917,-91.1085
2,e,s,t,c,buff,s,white,pendant,brown,numerous,meadows,-23.2831,-37.9087
3,p,yellow,t,c,brown,s,white,pendant,black,scattered,urban,-54.2210,108.3953


In [35]:
#converting string representation into an integer representation.

df_mushrooms['habitat']=df_mushrooms["habitat"].astype('category').cat.codes
df_mushrooms['cap-color']=df_mushrooms["cap-color"].astype('category').cat.codes
corr = df_mushrooms.corr()

print('Q2 Does habitat and cap-color correlate?')
print("No correlation between cap-color & habitat ")
print (corr)


Q2 Does habitat and cap-color correlate?
No correlation between cap-color & habitat 
           cap-color   habitat       lat       lon
cap-color   1.000000 -0.017502  0.010444 -0.002747
habitat    -0.017502  1.000000  0.007488 -0.003121
lat         0.010444  0.007488  1.000000  0.007516
lon        -0.002747 -0.003121  0.007516  1.000000


In [36]:
print("replacing '-' with '_' in dataframe column names")

df_mushrooms.columns= df_mushrooms.columns.str.replace(r'-', '_')


replacing '-' with '_' in dataframe column names


In [37]:
print("checking dataframe for changes")
df_mushrooms.head(2)



checking dataframe for changes


,cap_shape,cap_color,odor,gill_size,gill_color,stalk_color_above_ring,veil_color,ring_type,spore_print_color,population,habitat,lat,lon
0,p,2,t,c,brown,s,white,pendant,black,scattered,4,46.4504,62.4556
1,e,2,t,c,buff,s,white,pendant,brown,numerous,0,37.6917,-91.1085


In [73]:
print('Q3 Considering a specific geographical point, what colours should we be able to see in the 10 km around it?')

# loop through rows in dataframes

""" using the lat and lon of the row below as a reference.

cap-shape	cap-color	odor	gill-size	gill-color	stalk-color-above-ring	veil-color	ring-type	spore-print-color	population	habitat	
p	        s	        t	      c	        brown	    s	                    white	    pendant	     black	            scattered	urban	        

lat	    lon	    Time
46.4504	62.4556	2:24:56 PM

find mushrooms which are within 10km from the location of the refernce mushroom above and exclude reference mushroom

"""

# variables to hold reference lat & log
ref_lat = df_mushrooms['lat'].head(1)
ref_lon = df_mushrooms['lon'].head(1)

for row in df_mushrooms.itertuples():
    # mushooms within 10km of reference mushoom location
    if (haversine((ref_lat, ref_lon ),(row.lat, row.lon))) <= 10 and row.lat != 46.4504:
        print (row.index, row.lat, row.lon, row.spore_print_color,  (haversine((ref_lat, ref_lon),(row.lat, row.lon))))
     


Q3 Considering a specific geographical point, what colours should we be able to see in the 10 km around it?
